**Mounting the colab on drive**


In [2]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Hate Video task'
assert FOLDERNAME is not None, "[!] Enter the foldername."

Mounted at /content/drive


In [3]:
!pip install noisereduce
import cv2
import math
from tqdm import tqdm
from glob import glob
import numpy as np
import pandas as pd
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import TensorDataset, DataLoader
from statistics import mode
import librosa
import noisereduce as nr
%matplotlib inline
dtype = torch.cuda.FloatTensor
device = torch.device("cuda:0")

  Created wheel for noisereduce: filename=noisereduce-1.1.0-cp36-none-any.whl size=7610 sha256=d76e02ea4a3ff0a3b72fc9b2e6d7183ab06866a268a63622ea56960b7a5f28b0
  Stored in directory: /root/.cache/pip/wheels/a6/2c/70/f9ccb41280dcfbe3eaeb7930f913dd85394617f3d3136f29cc
Successfully built noisereduce


**Loading the base model and defining architecture for main classification model and loading saved weights for the model**

In [4]:
Base_model = models.vgg16(pretrained = True)
Base_model.classifier = Base_model.classifier[:-7]

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [5]:
Base_model.to(device)
Base_model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
v_model = nn.Sequential(
    nn.Linear(7*7*512, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 2),
    nn.Softmax(dim = 1)
).to(device)
print(v_model.parameters)

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=25088, out_features=1024, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=512, out_features=256, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.5, inplace=False)
  (9): Linear(in_features=256, out_features=128, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.5, inplace=False)
  (12): Linear(in_features=128, out_features=2, bias=True)
  (13): Softmax(dim=1)
)>


In [7]:
PATH = '/content/drive/My Drive/Hate Video task/Models/Model1.pt'
v_model.load_state_dict(torch.load(PATH))
v_model.to(device)
v_model.eval()

Sequential(
  (0): Linear(in_features=25088, out_features=1024, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=512, out_features=256, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.5, inplace=False)
  (9): Linear(in_features=256, out_features=128, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.5, inplace=False)
  (12): Linear(in_features=128, out_features=2, bias=True)
  (13): Softmax(dim=1)
)

In [8]:
a_model = nn.Sequential(
    nn.Linear(257,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,2),
    nn.Softmax(dim = 1)
).to(device)

In [9]:
PATH = '/content/drive/My Drive/Hate Video task/Models/Model2.pt'
a_model.load_state_dict(torch.load(PATH))
a_model.to(device)
a_model.eval()

Sequential(
  (0): Linear(in_features=257, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Dropout(p=0.5, inplace=False)
  (5): Linear(in_features=128, out_features=64, bias=True)
  (6): ReLU()
  (7): Linear(in_features=64, out_features=2, bias=True)
  (8): Softmax(dim=1)
)

**Processing the dataset and calculating accuracy**

In [10]:
def process_audio(audio_data):
  audio_data = nr.reduce_noise(audio_clip=audio_data, noise_clip=audio_data, verbose=False)
  trimmed, index = librosa.effects.trim(audio_data, top_db=20, frame_length=512, hop_length=64)
  stft = np.abs(librosa.stft(trimmed, n_fft=512, hop_length=256, win_length=512))
  stft = np.mean(stft, axis = 1, keepdims = True)
  return stft

In [ ]:
predicted = []
actual = []

false_positive = 0
true_positive = 0
false_negative = 0
true_negative = 0

video_folder = '/content/drive/My Drive/Hate Video task/Dataset/Test_video/'
audio_folder = '/content/drive/My Drive/Hate Video task/Dataset/Test_audio/'
i = 0
for entry in os.listdir(video_folder):
  print("current iter = {}".format(i))
  i+=1
  file_name = video_folder + entry
  cap = cv2.VideoCapture(file_name)
  frame_rate = cap.get(5)
  X = []
  while(cap.isOpened()):
    frame_id = cap.get(1)
    ret, frame = cap.read()
    if(ret!=True):
      break
    if(frame_id%math.floor(frame_rate)==0):
      X.append(frame)
  cap.release()
  X = np.array(X)
  try:
    X = np.transpose(X, [0, 3, 1, 2])
  except:
    continue
  tensor_x = torch.Tensor(X)
  del X
  tensor_y = torch.randn(tensor_x.shape[0],1)
  tensor_x = tensor_x.to(device)
  tensor_x = tensor_x.type(torch.cuda.FloatTensor)
  test_fin_dataset = TensorDataset(tensor_x,tensor_y)
  test_dataloader = DataLoader(test_fin_dataset, batch_size = 4)

  video_score = []

  for x,y in test_dataloader:
    x = Base_model(x)
    scores = v_model(x)
    scores = scores.tolist()
    video_score += scores

  audio_X = []
  audio_name = audio_folder + entry.split('.')[0] + '.wav'
  audio_x, sr = librosa.load(audio_name)
  audio_x = process_audio(audio_x)
  audio_X.append(audio_x)

  tensor_audio_x = torch.Tensor(audio_X)
  dim = tensor_audio_x.shape
  tensor_audio_x = torch.reshape(tensor_audio_x, (dim[0],dim[1]))
  tensor_audio_x = tensor_audio_x.to(device).type(torch.cuda.FloatTensor)

  fin_audio_score = a_model(tensor_audio_x).tolist()
  lm = 5
  fin_score = video_score + lm * fin_audio_score
  fin_score = torch.max(torch.Tensor(fin_score),1)
  pred = 0
  try:
    pred = mode(fin_score.indices.tolist())
  except:
    pred = 0
  act = 0
  if entry[0]=='h':
    act = 1
  else:
    act = 0
  if act==pred==1:
    true_positive += 1
  elif act==pred==0:
    true_negative += 1
  elif act==1 and pred == 0:
    false_negative += 1
  else:
    false_positive += 1
  predicted.append(pred)
  actual.append(act)
  torch.cuda.empty_cache()
  print("Actual value = {} and Predicted value = {}".format(act, pred))

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy of model is: -")
print(accuracy_score(predicted, actual)*100)

In [ ]:
print("True positives = {}; False positives = {}; True negatives = {}; False negatives = {}".format(true_positive, false_positive, true_negative, false_negative))